In [1]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [2]:
host='localhost'
port=9999

In [3]:
# ! nc localhost 9999

In [3]:
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Step1_1-NetCat") \
    .config("spark.executor.memory", "512mb") \
    .getOrCreate()

21/10/06 07:46:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark.sparkContext.setLogLevel("ERROR")


In [5]:
lines = spark\
        .readStream\
        .format('socket')\
        .option('host', host)\
        .option('port', port)\
        .load()


In [7]:
lines.printSchema()

root
 |-- value: string (nullable = true)



In [8]:
words_df = lines.select(
        explode(
            split(lines.value, ' ')
        ).alias('word')
    )

In [9]:
wordCounts = words_df.groupBy('word')\
                      .count()

In [ ]:
query = wordCounts.writeStream\
                      .outputMode('complete')\
                      .format('console')\
                      .start()\
                      .awaitTermination()


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|World|    1|
|   Hi|    1|
|Hello|    1|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|World|    3|
|   Hi|    1|
|Hello|    2|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|World|    3|
|   Hi|    1|
|Hello|    2|
| TEST|    1|
+-----+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|World|    3|
|   Hi|    1|
|Hello|    2|
| TEST|    2|
| test|    1|
+-----+-----+



In [11]:
spark.stop()